In [3]:
import Pude_training_loop.loss_functions_torch as loss_functions
import Pude_training_loop.pude_utils as pude_utils
import Pude_training_loop.model_training as model_training
import monocular_stereo_matching.stereo_dataset_loader as data_loader
from monocular_stereo_matching.stereo_pair_gen import Stereo_Pair_Generator
import torch
from PIL import Image
import pickle
import numpy as np
import os

import monocular_stereo_matching.stereo_eval as stereo_eval

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_path = "new_unimatch_model_pude_AdamW_beta_60_600_alphas_4_0_4_epochs_3_lr_1e-6.pth"
model_path ="new_unimatch_model_Adamw_betas_60_600,_alphas_4_0_4_checkpoint_epoch_0_finished.pth"
# model_path = "new_unimatch_model_cosine_scheduler_AdamW_betas_100_50,_alphas_4_0-4_4_lr_5e-6_epochs_3.pth"
# model_path = "new_unimatch_model_cosine_scheduler_AdamW_betas_60_60,_alphas_4_0-4_8_lr_5e-6_epochs_3.pth"
# model_path = "new_unimatch_model_cosine_scheduler_AdamW_betas_200_100,_alphas_4_0-4_4_lr_5e-6_epochs_3.pth"
# datasets = ["canyon1", "canyon2", "rock_garden1", "rock_garden2"]
datasets = ["canyon2"]
for dataset_name in datasets:
    for i in range(1,1+1):
        # model_path =  f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_100_80_10_alphas_4_0-4_4_lr_5e-6_epochs_{i}.pth"
        # model_path =  "new_unimatch_model_cosine_scheduler_AdamW_betas_300_400,_alphas_4_0-4_4_lr_5e-6_epochs_3.pth"
        model_path_nums = f"new_combined_60_600_epoch_{i}"
        # create the results folder if it doesnt exist
        if not os.path.exists(f"Results/FLSea/{dataset_name}/new_unimatch_{model_path_nums}/images"):
            os.makedirs(f"Results/FLSea/{dataset_name}/new_unimatch_{model_path_nums}/images")
            os.makedirs(f"Results/FLSea/{dataset_name}/combined_{model_path_nums}/images")
        if not os.path.exists(f"Results/FLSea/{dataset_name}/org_unimatch/images"):
            os.makedirs(f"Results/FLSea/{dataset_name}/org_unimatch/images")



        org_unimatch_model, org_unimatch_image_processor = model_training.get_model_image_processor_pair(model_name="unimatch", model_path=model_training.models["unimatch"], device=device)
        new_unimatch_model, new_unimatch_image_processor = model_training.get_model_image_processor_pair(model_name="unimatch", model_path=model_path, device=device)

        
        dataset_names = [dataset_name]
        save_ground_truth = False

        dataset_loader = data_loader.DatasetLoader(total_num_pairs=100, dataset_names=dataset_names) # Initialize dataset loader with default parameters
        stereo_pair_gen = Stereo_Pair_Generator(image_dim=model_training.default_image_dim)
        stereo_evaluator = stereo_eval.StereoEval()


        new_unimatch_results_path = f"Results/FLSea/{dataset_name}/new_unimatch_{model_path_nums}/images"
        org_unimatch_results_path = f"Results/FLSea/{dataset_name}/org_unimatch/images"
        combined_grid_path = f"Results/FLSea/{dataset_name}/combined_{model_path_nums}/images"

        # seed the torch
        torch.manual_seed(model_training.seed)
        np.random.seed(model_training.seed)


        new_evals = np.zeros(len(stereo_evaluator.criteria_functions))
        org_evals = np.zeros(len(stereo_evaluator.criteria_functions))
        total_images = len(dataset_loader)

        orig_unimatch_results = []

        def get_model_output(unimatch_model, unimatch_image_processor, input, ground_truth, valid_indices, device):
            unimatch_output = model_training.get_model_output(model=unimatch_model, 
                                                            image_processor=unimatch_image_processor, 
                                                            raw_image=input, 
                                                            device=device, requires_grad=False)
            unimatch_output = unimatch_output.cpu().detach().numpy().squeeze()
            ground_truth_image = pude_utils.get_depth_as_image(ground_truth)
            model_output_image = pude_utils.get_depth_as_image(unimatch_output)
            diff_image = np.zeros_like(ground_truth)
            diff_image[valid_indices] = np.abs(ground_truth[valid_indices]-unimatch_output[valid_indices])
            abs_difference_image = pude_utils.get_depth_as_image(diff_image)
            image_grid = pude_utils.make_image_grid_title(images=[Image.fromarray(unimatch_input['image1']), ground_truth_image, model_output_image, abs_difference_image], 
                                                titles=["Left Image", "Ground Truth", "Model Output", "Absolute Difference"],
                                                rows=1, cols=4)
            evals = stereo_evaluator.evaluate(ground_truth[valid_indices], unimatch_output[valid_indices])
            return unimatch_output, model_output_image, evals, image_grid


        for i in range(total_images):
            left_image, right_image, ground_truth, valid_indices = dataset_loader[i]
            # left_image = torch.tensor(left_image)
            # right_image = torch.tensor(right_image)

            
            unimatch_input = {"image1": left_image, "image2": right_image}
            org_output, org_output_image, org_evals, org_image_grid = get_model_output(org_unimatch_model, 
                                                                                    org_unimatch_image_processor, 
                                                                                    unimatch_input, 
                                                                                    ground_truth,
                                                                                        valid_indices, device)
            new_output, new_output_image, new_evals, new_image_grid = get_model_output(new_unimatch_model, 
                                                                                    new_unimatch_image_processor, 
                                                                                    unimatch_input, 
                                                                                    ground_truth,
                                                                                    valid_indices, device)

            orig_unimatch_results.append(org_output)

            diff_image = pude_utils.get_depth_as_image(np.abs(org_output-new_output))
            diff_grid = pude_utils.make_image_grid_title(images=[Image.fromarray(left_image), org_output_image, new_output_image, diff_image],
                                                            titles=["Left Image", "Org Output", "New Output", "Absolute Difference"],
                                                            rows=1, cols=4)
            combined_grid = pude_utils.make_image_grid_title(images=[org_image_grid, new_image_grid, diff_grid],
                                                            titles=["Org Unimatch", "New Unimatch", "Difference"],
                                                            rows=3, cols=1)
            if save_ground_truth:
                org_image_grid.save(f"{org_unimatch_results_path}/image_{i}.png")
            new_image_grid.save(f"{new_unimatch_results_path}/image_{i}.png")
            combined_grid.save(f"{combined_grid_path}/image_{i}.png")


        new_evals/=total_images
        org_evals/=total_images

        print(f"-------------------{dataset_name} Results-------------------")
        print("original unimatch: ")
        print(stereo_evaluator.criteria_functions.keys())
        print(org_evals)
        print(f"{model_path[:-4]}: ")
        print(stereo_evaluator.criteria_functions.keys())
        print(new_evals)
        print("------------------------------------------------------------")
        if save_ground_truth:
            with open(f"Results/FLSea/{dataset_name}/orig_unimatch_results.pkl", "wb") as f:
                pickle.dump(orig_unimatch_results, f)

-------------------canyon2 Results-------------------
original unimatch: 
dict_keys(['abs_rel_error', 'squared_rel_error', 'rmse', 'rmse_log', 'silog', 'accuracy_1.05', 'accuracy_1.05^2', 'accuracy_1.05^3'])
[0.00736869 0.37473892 0.0219572  0.00614564 0.00478607 0.00100505
 0.00245349 0.00358246]
new_unimatch_model_Adamw_betas_60_600,_alphas_4_0_4_checkpoint_epoch_0_finished: 
dict_keys(['abs_rel_error', 'squared_rel_error', 'rmse', 'rmse_log', 'silog', 'accuracy_1.05', 'accuracy_1.05^2', 'accuracy_1.05^3'])
[0.00491775 0.00985837 0.02150267 0.00463749 0.00352757 0.00116772
 0.00234545 0.00307678]
------------------------------------------------------------
